# Context

## Title

**Abstract:**
One-sentence description

**Description:**
In the following cell, I...


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
from matplotlib import pyplot as plt

from src import NATURE_PALETTE as nature_colors

## Yellow River Breaks

**Abstract:**
One-sentence description

**Description:**
In the following cell, I...


In [ ]:
outage = pd.read_excel(r"../data/source/river break.xlsx")

PERIODS = {
    1978: "start",
    1987: "87-WAS",
    1998: "98-UBR",
    2008: "end",
}


def plot_outage(ax=None):
    if not ax:
        fig, ax = plt.subplots(figsize=(9, 2))
    bubbles = ax.scatter(
        x=outage["年份"],
        y=outage["断流天数"],
        s=outage["断流长度"] * 0.6,
        color="lightgray",
        edgecolors=nature_colors["Nature"],
        linewidth=1.5,
        alpha=0.9,
        label="Drying-up",
        zorder=1,
    )
    ax.set_xlim(1970, 2010)
    ax.set_ylim(-10, 170)
    #     ax.axvline(1987, ls=':', color=nature_colors['NS'], lw=3, label='Policy 1', zorder=0)
    #     ax.axvline(1998, ls=':', color=nature_colors['NG'], lw=3, label='Policy 2', zorder=0)
    #     ax.axvline(1978, ls='-.', color='gray', lw=1, label='Study period division')
    #     ax.axvline(2008, ls='-.', color='gray', lw=1)
    ax.spines["top"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["bottom"].set_visible(True)

    # ax.axvspan(1975, 1987, alpha=0.1, label='Structure 1')
    # ax.axvspan(1987, 1998, alpha=0.1, color='red', label='Mismatched institution')
    # ax.axvspan(1998, 2008, alpha=0.1, color='green', label='Structure 3')

    ax.set_xlabel("Year")
    ax.set_ylabel("Drying-up / days")
    return bubbles


fig, ax = plt.subplots()
scatter = plot_outage(ax=ax)
handles, labels = scatter.legend_elements(prop="sizes", alpha=0.9)
legend2 = ax.legend(handles, labels, loc="upper right", title="Sizes")
plt.show();

## Drought

**Abstract:**
One-sentence description

**Description:**
In the following cell, I...


In [ ]:
drought = pd.read_excel(r"../data/source/干旱严重程度.xlsx", index_col=0)
drought.columns = [f"m{i}" for i in (1, 3, 6, 12)]
col = "m1"


def plot_drought(col, ax=None):
    if not ax:
        fig, ax = plt.subplots(figsize=(9, 2))
    mean_10 = drought.rolling(window=10, min_periods=10, center=True).mean()[col]
    # drought[col].plot.bar(ax=ax)
    ax.bar(
        x=drought[col].index,
        height=drought[col].values,
        color="#e0a418",
        alpha=0.8,
        label="Drought",
        zorder=0,
    )
    ax.plot(
        mean_10.index,
        mean_10.values,
        ls="-.",
        lw=2,
        color=nature_colors["NS"],
        label="10yrs-Avg. drought index",
    )
    ax.set_xlim(1970, 2010)
    ax.axhline()
    ax.spines["top"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["bottom"].set_visible(True)


plot_drought(col)
plt.show();

In [ ]:
import scipy

droughts_p1 = drought["m1"].loc[1988:1998]
droughts_p2 = drought["m1"].loc[1998:2008]
t, pval = scipy.stats.ttest_ind(droughts_p1, droughts_p2)

print("Differences between Droughts 1988~1998; 1998~2008:\n")
droughts_p1.mean()
droughts_p2.mean()
t
pval

In [ ]:
from src.multi_synth import MultiSynth

exp87 = MultiSynth.load_from_pickle("../model/model_87.pkl")
k_87 = exp87.plot_pre_post()

exp98 = MultiSynth.load_from_pickle("../model/model_98.pkl")
k_98 = exp98.plot_pre_post()

print("1998-UBR 之后的速度: ")
print(k_98)
plt.show();

In [ ]:
data = exp87.agg_results("sum").loc[exp87.treated_time :, :]
observation = data["Origin"].sum()
estimation = data["Synth"].sum()
ratio = 100 * (observation - estimation) / estimation
print(
    f"From {exp87.treated_time} to {exp87.end}, the observed water use of the YRB provinces reached ${observation:.2f} km^3$ in sum while the estimation of water use only suggest ${estimation:.2f} km^3$, ${ratio:.2f}\%$ increased."
)

## Summary Plot

**Abstract:**
One-sentence description

**Description:**
In the following cell, I...


In [ ]:
fig = plt.figure(figsize=(9, 5), constrained_layout=True)

# prepare plotting
gs = fig.add_gridspec(
    ncols=3, nrows=2, hspace=0.0, height_ratios=[1.2, 1], width_ratios=[7, 10, 10]
)

ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[0, 2])
ax4 = fig.add_subplot(gs[1, :])
ax3 = ax4.twinx()

# plots
exp87.plot_pre_post(ax=ax1, axvline=False)
exp98.plot_pre_post(ax=ax2, axvline=False)
bubbles = plot_outage(ax=ax3)
plot_drought("m1", ax4)
ax1.legend_.remove()
ax2.legend_.remove()
# beauty & annotation

ax1.axvline(1987, ls="--", color=nature_colors["NS"], lw=3, zorder=0)
ax2.axvline(1998, ls="--", color=nature_colors["NG"], lw=3, zorder=0)

ax4.set_ylabel("Drought index")
ax3.annotate(
    "",
    xy=(1978, ax3.get_ylim()[1]),
    xycoords="data",
    xytext=(1998, ax3.get_ylim()[1]),
    textcoords="data",
    arrowprops=dict(
        arrowstyle="<->",
        connectionstyle="arc3",
        color=nature_colors["NS"],
    ),
)
ax3.annotate(
    "",
    xy=(1987, 150),
    xycoords="data",
    xytext=(2008, 150),
    textcoords="data",
    arrowprops=dict(
        arrowstyle="<->",
        connectionstyle="arc3",
        color=nature_colors["NG"],
    ),
)

ax4.axvline(1987, ls="--", color=nature_colors["NS"], lw=3, label="87-WAS", zorder=0)
ax4.axvline(1998, ls="--", color=nature_colors["NG"], lw=3, label="98-UBR", zorder=0)
ax4.axvline(1978, ls="-.", color="gray", lw=1, label="Study period division")
# ax3.text("")
legend_handles = []
legend_labels = []
for handle, label in zip(*ax2.get_legend_handles_labels()):
    legend_handles.append(handle)
    legend_labels.append(label)
for handle, label in zip(*ax3.get_legend_handles_labels()):
    legend_handles.append(handle)
    legend_labels.append(label)
for handle, label in zip(*ax4.get_legend_handles_labels()):
    legend_handles.append(handle)
    legend_labels.append(label)

ax3.text(
    1986.5,
    155,
    "IS1: 87-WAS",
    color=nature_colors["NS"],
    horizontalalignment="right",
    weight="bold",
)
ax3.text(
    1998.5,
    155,
    "IS2: 98-UBR",
    color=nature_colors["NG"],
    horizontalalignment="left",
    weight="bold",
)


# 使用图片的比例来定位
def get_position_by_ratio(ax, x_ratio, y_ratio):
    """
    使用图片的比例来返回定位，从而更好的控制说明文字的位置
    ax: 一个 matplotlib 的画图轴对象
    x_ratio: 横坐标的比例位置
    y_ratio: 纵坐标的比例位置
    """
    x_min, x_max = ax.get_xlim()
    y_min, y_max = ax.get_ylim()
    x = (x_max - x_min) * x_ratio + x_min
    y = (y_max - y_min) * y_ratio + y_min
    return x, y


for ax in [ax1, ax2]:
    ax.set_yticks(range(70, 111, 10))

for i, ax in enumerate([ax1, ax2, ax3]):
    ax.grid(True, color="lightgray", ls=":")
    ax.tick_params(direction="in")
    x, y = get_position_by_ratio(ax, 0.05, 0.95)
    ax.text(x, y, ("A", "B", "C")[i], weight="bold", fontsize=12)

legend = fig.legend(
    loc="upper left",
    handles=legend_handles,
    labels=legend_labels,
    handletextpad=1.5,
    handleheight=1.5,
    markerscale=0.7,
)
plt.savefig("../figs/outputs/main_results2.pdf", format="pdf")
plt.show();

In [ ]:
fig = exp87.panel_plots()
# plt.savefig("figs/outputs/87panel.pdf", format='pdf')

In [ ]:
fig = exp87.panel_plots("in-space placebo", in_space_exclusion_multiple=3)
# plt.savefig("figs/outputs/87placebo.pdf", format='pdf')

In [ ]:
fig = exp98.panel_plots()
# plt.savefig("figs/outputs/98panel.pdf", format='pdf')

In [ ]:
fig = exp98.panel_plots("in-space placebo", in_space_exclusion_multiple=3)
# plt.savefig("figs/outputs/98placebo.pdf", format='pdf')

In [ ]:
fig = exp87.panel_plots("rmspe ratio")

In [ ]:
fig = exp98.panel_plots("rmspe ratio")

In [ ]:
def rmse_report(exp, caption):
    result = {}
    for unit in exp.units:
        sc = exp.units.get(unit).model
        data = sc.original_data
        df = data.rmspe_df
        result[unit] = df[df["unit"] == unit].drop("unit", axis=1).iloc[0, :]
    median = df[df["unit"] != unit]["post/pre"].median()
    result = pd.DataFrame(result)
    result = result.T
    result["Significant"] = result["post/pre"] > median
    result.columns = pd.MultiIndex.from_tuples([(f"{caption}", col) for col in result])
    return result


rmspe_87 = rmse_report(exp87, "1987-WAS")
rmspe_98 = rmse_report(exp98, "1998-UBR")

concated = pd.concat([rmspe_87, rmspe_98], axis=1)
concated.style.format("{:.3f}")
concated.style.format({("1987-WAS", "Significant"): bool})
concated.style.format({("1998-UBR", "Significant"): bool})
concated.to_csv("compare_RMSE.csv")

## Compare 87-98 regulating

**Abstract:**
One-sentence description

**Description:**
In the following cell, I...


In [ ]:
def add_statistic_items(exp):
    def extract_mean_std(data, start, end):
        data = data.loc[start:end, :]
        return data.mean(), data.std()

    statistic = exp.get_statistic_df()
    start, end = exp.treated_time, exp.end
    statistic["Total_WU"] = extract_mean_std(wu_total, start, end)[0]
    statistic["YR_WU"] = extract_mean_std(wu_yr, start, end)[0]
    statistic["ratio"] = extract_mean_std(ratio, start, end)[0]

    statistic["scheme83"] = quota.loc[1983, :]
    statistic["scheme87"] = quota.loc[1987, :]
    statistic["plan"] = plan.loc["Sum", :]
    statistic["satisfied"] = statistic["scheme87"] / statistic["plan"]
    statistic["unsatisfied"] = 1 - statistic["satisfied"]
    statistic["stress"] = statistic["unsatisfied"] * statistic["YR_WU"]

    statistic["diff_ratio"] = statistic["diff_sum"] / abs(statistic["diff_sum"].sum())
    statistic["punished"] = statistic["scheme83"] - statistic["scheme87"]

    statistic = statistic.sort_values("stress", ascending=False)
    return statistic


wu_yr_path = "../data/processed/wu_yr.csv"
wu_all_path = "../data/processed/wu_all.csv"
ratio_path = "../data/processed/ratio.csv"
plan_path = "../data/processed/plan.csv"
quota_path = "../data/processed/quota.csv"

wu_yr = pd.read_csv(wu_yr_path, index_col=0)
wu_total = pd.read_csv(wu_all_path, index_col=0)
ratio = pd.read_csv(ratio_path, index_col=0)
plan = pd.read_csv(plan_path, index_col=0)
quota = pd.read_csv(quota_path, index_col=0)

statistic_87 = add_statistic_items(exp87)
statistic_98 = add_statistic_items(exp98)

In [ ]:
test = statistic_87["YR_WU"] + statistic_98["YR_WU"]
test / test.sum()
base = statistic_87["scheme87"] / statistic_87["scheme87"].sum()

In [ ]:
statistic_98

In [ ]:
from src.plots import correlation_analysis

correlation_analysis(statistic_98, xs=["YR_WU", "satisfied"], y="diff_ratio")

In [ ]:
compare_df = pd.DataFrame()
compare_df["87_ratio"] = statistic_87["diff_ratio"]
compare_df["98_ratio"] = statistic_98["diff_ratio"]
compare_df["wu_ratio_87"] = statistic_87["YR_WU"] / statistic_87["YR_WU"].sum()
compare_df["wu_ratio_98"] = statistic_98["YR_WU"] / statistic_98["YR_WU"].sum()
compare_df["YR_WU"] = (compare_df["wu_ratio_87"] + compare_df["wu_ratio_98"]) / 2
compare_df = compare_df.sort_values("YR_WU", ascending=False)

In [ ]:
from matplotlib import ticker
import numpy as np

plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.direction"] = "in"
fig, ax1 = plt.subplots(figsize=(7, 4))

width = 0.36
x = np.arange(8)
index = compare_df.index
ax2 = ax1.twinx()

ax2.bar(
    x - width / 2,
    width=width,
    height=compare_df["wu_ratio_87"],
    color="lightgray",
    alpha=0.4,
    edgecolor="white",
    hatch="xxx",
    label="WU ratio",
    zorder=0,
)
ax2.bar(
    x + width / 2,
    width=width,
    height=compare_df["wu_ratio_98"],
    color="lightgray",
    alpha=0.4,
    edgecolor="white",
    hatch="xxx",
    zorder=0,
)
ax2.scatter(
    range(8),
    base[compare_df.index],
    marker="^",
    color="red",
    edgecolor="white",
    s=80,
    label="Quota",
)

ax1.bar(
    x - width / 2,
    width=width,
    height=compare_df["87_ratio"],
    color="#C83C1C",
    alpha=0.8,
    zorder=1,
    label="87-WAS",
    edgecolor="white",
)
ax1.bar(
    x + width / 2,
    width=width,
    height=compare_df["98_ratio"],
    color="#006C43",
    alpha=0.8,
    zorder=1,
    label="98-UBR",
    edgecolor="white",
)


# 坐标轴距离
ax1.set_yticks(np.arange(-0.5, 0.51, 0.25))
ax1.set_ylim(-0.5, 0.5)
ax1.set_xlim(-0.8, 7.8)
ax2.set_yticks(np.arange(-0.4, 0.41, 0.1))
ax2.set_ylim(-0.4, 0.4)
ax2.set_yticklabels(["", "", "", "", "0%", "10%", "20%", "30%", "40%"])

for ax in [ax1, ax2]:
    ax.spines["top"].set_visible(False)
    ax.spines["left"].set_visible(True)
    ax.spines["right"].set_visible(True)
    ax.spines["bottom"].set_visible(False)

xticklabels = index.to_list()
xticklabels.insert(0, "")
ax1.set_xticklabels(xticklabels)
claen_labels = []
for label in ax1.get_xticklabels():
    text = label.get_text()
    if text == "Neimeng":
        text = "Inner\nMongolia"
    elif text == "Shanxi":
        text = "Shaanxi"
    elif text == "Shaanxi":
        text = "Shanxi"
    claen_labels.append(text)
ax1.set_xticklabels(claen_labels)
ax1.set_ylabel("Extra WU over the estimation")
ax2.set_ylabel("WU ratio")
#### 辅助线 ========
ax1.axhline(0, ls="--", color="gray", lw=1, zorder=0)
ax1.annotate(
    "",
    xy=(0 - width, -0.4),
    xycoords="data",
    xytext=(3 + width, -0.4),
    textcoords="data",
    arrowprops=dict(arrowstyle="<->", connectionstyle="arc3", color="black", lw=1.5),
)
ax1.text(
    1.5,
    -0.42,
    "Major water users",
    color="black",
    horizontalalignment="center",
    verticalalignment="top",
)
ax1.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1, decimals=0))

# -=====  图例

legend_handles = []
legend_labels = []
for handle, label in zip(*ax1.get_legend_handles_labels()):
    legend_handles.append(handle)
    legend_labels.append(label)

for handle, label in zip(*ax2.get_legend_handles_labels()):
    legend_handles.append(handle)
    legend_labels.append(label)

fig.legend(
    loc=(0.7, 0.73),
    frameon=False,
    handles=legend_handles,
    labels=legend_labels,
    handletextpad=1.5,
)
plt.savefig("../figs/outputs/fig3.pdf", format="pdf")
plt.savefig("../figs/outputs/fig3.jpg", format="jpg", dpi=300)
plt.show();

In [ ]:
compare_df["87_ratio"].iloc[:4].mean()
compare_df["98_ratio"]

In [ ]:
compare_df["98_ratio"][compare_df["98_ratio"] < 0].mean()

In [ ]:
compare_df.loc["Neimeng", :]

In [ ]:
(
    compare_df.loc["Neimeng", "98_ratio"] - compare_df.loc["Neimeng", "87_ratio"]
) / compare_df.loc["Neimeng", "87_ratio"]